In [1]:
import pandas as pd 
from datasets import load_dataset
import numpy as np
import os
import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")
dataset


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
#merge train test and validation
train = dataset['train']
test = dataset['test']
validation = dataset['validation']

In [6]:
df = pd.concat([train.to_pandas(), test.to_pandas(), validation.to_pandas()], ignore_index=True)
df = df.drop_duplicates(subset='article')
df.head()

KeyboardInterrupt: 

: 

In [8]:
val = dataset['validation']
val = pd.DataFrame(val)
val.head()

,article,highlights,id
0,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...,a4942dd663020ca54575471657a0af38d82897d6
1,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend .\nLea...,4157bc4da185971e2742f349d69a037343bc0d95
2,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...,60736693e3b1b32d14337a317190c6606e879a85
3,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...,8cdf9cc3ed0276b7a7944cc18ba459355b5984ad
4,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ...",2e6613d531843515bf5401286cc3e45c4df530d2


In [9]:
test = dataset['test']
test = pd.DataFrame(test)
test.head()

,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, \
    Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
"""
Preprocessing:
# Cleaning the Data
    # Lemmetization
    # Remove Stop Words
    
# Determining the Maximum Permissible Sequence Lengths
# Selecting Plausible Texts and Summaries
# Tokenizing the Text
# Removing Empty Text and Summaries
"""

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

# Model Desctiption
The proposed model introduces an advanced text summarization framework leveraging deep learning techniques to mitigate common seq2seq limitations, such as repetitiveness and OOV issues. The approach begins with comprehensive preprocessing, including lemmatization, stop-word removal, and tokenization, to refine input documents. Subsequently, it employs the GloVe toolkit for efficient word vectorization, enabling enhanced representation of textual data. The core of the model utilizes a combination of bidirectional LSTM for encoding and unidirectional LSTM for decoding, integrated within a TensorFlow framework. This setup is further optimized using cross-entropy loss and the Adam optimizer, aiming to improve summary quality and accuracy. By addressing seq2seq's inherent drawbacks, this model strives for superior performance in abstractive text summarization.

In [3]:
def clean(df):
  """
  Cleans the input dataframe by using lamentization, stop words removal, etc.

  Args:
  df: Dataframe to be cleaned
  """
  # Lemmetization
  lemmatizer = WordNetLemmatizer()
  df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))
  df['summary'] = df['summary'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

  # Remove Stop Words
  stop_words = set(stopwords.words('english'))
  df['text'] = df['text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))
  df['summary'] = df['summary'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

  return df


In [ ]:
def cleans(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    text = [lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stop_words]
    return text
